# Calculate the differences between graphs in service0001 and service0002

In [127]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [128]:
import os, sys

In [129]:
import osmnx as ox
import pandas as pd
import geopandas as gpd
import networkx as nx
import numpy as np
from shapely.geometry import Point

In [130]:
# Get reference to GOSTNets
import sys
sys.path.append(r'C:\repos\GOSTnets')
import GOSTnets as gn

## Load graphs

In [132]:
# read back your graphs from step 2 from you saved pickle
# https://github.com/geopandas/geopandas/issues/1234
service0001_edges_gdf = gpd.read_file(r"temp/gtfs_export_cap_haitien_service0001_edges.csv", GEOM_POSSIBLE_NAMES="geometry", KEEP_GEOM_COLUMNS="NO") 
service0002_edges_gdf = gpd.read_file(r"temp/gtfs_export_cap_haitien_service0002_edges.csv", GEOM_POSSIBLE_NAMES="geometry", KEEP_GEOM_COLUMNS="NO")

In [133]:
service0001_edges_gdf

,field_1,stnode,endnode,mode,length,trips,geometry
0,0,GLYN8_stop_0119,GLYN8_stop_0096,transit,2119.0,1,"LINESTRING (-71.73141 19.54854, -71.94963 19.6..."
1,1,GLYN8_stop_0143,GLYN8_stop_0260,transit,1462.0,1,"LINESTRING (-72.20758 19.74893, -72.27349 19.7..."
2,2,GLYN8_stop_0143,GLYN8_stop_0081,transit,74.0,2,"LINESTRING (-72.20758 19.74893, -72.20682 19.7..."
3,3,GLYN8_stop_0143,GLYN8_stop_0154,transit,122.0,1,"LINESTRING (-72.20758 19.74893, -72.20663 19.7..."
4,4,GLYN8_stop_0176,GLYN8_stop_0074,transit,706.0,1,"LINESTRING (-72.24224 19.78266, -72.22709 19.7..."
...,...,...,...,...,...,...,...
337,337,GLYN8_stop_0159,GLYN8_stop_0125,transit,177.0,1,"LINESTRING (-72.17812 19.69676, -72.18074 19.7..."
338,338,GLYN8_stop_0156,GLYN8_stop_0123,transit,78.0,1,"LINESTRING (-72.19765 19.62419, -72.19785 19.6..."
339,339,GLYN8_stop_0124,GLYN8_stop_0035,transit,122.0,1,"LINESTRING (-72.15876 19.69403, -72.14931 19.6..."
340,340,GLYN8_stop_0132,GLYN8_stop_0046,transit,364.0,2,"LINESTRING (-72.20224 19.75554, -72.20458 19.7..."


In [134]:
service0002_edges_gdf

,field_1,stnode,endnode,mode,trips,length,geometry
0,0,B9IOC_stop_0001,B9IOC_stop_0139,transit,1,2905.0,"LINESTRING (-72.18084 19.70481, -71.82658 19.6..."
1,1,B9IOC_stop_0001,B9IOC_stop_0219,transit,3,150.66666666666666,"LINESTRING (-72.18084 19.70481, -72.18008 19.6..."
2,2,B9IOC_stop_0002,B9IOC_stop_0075,transit,6,32.666666666666664,"LINESTRING (-72.21223 19.74409, -72.21330 19.7..."
3,3,B9IOC_stop_0003,B9IOC_stop_0152,transit,3,49.333333333333336,"LINESTRING (-72.20667 19.75217, -72.20703 19.7..."
4,4,B9IOC_stop_0003,B9IOC_stop_0023,transit,3,67.66666666666667,"LINESTRING (-72.20667 19.75217, -72.20743 19.7..."
...,...,...,...,...,...,...,...
355,355,B9IOC_stop_0096,B9IOC_stop_0126,transit,1,465.0,"LINESTRING (-71.94963 19.63536, -71.99543 19.6..."
356,356,B9IOC_stop_0139,B9IOC_stop_0157,transit,1,987.0,"LINESTRING (-71.82658 19.63542, -71.73564 19.5..."
357,357,B9IOC_stop_0157,B9IOC_stop_0119,transit,1,292.0,"LINESTRING (-71.73564 19.54705, -71.73141 19.5..."
358,358,B9IOC_stop_0088,B9IOC_stop_0116,transit,1,243.0,"LINESTRING (-72.28491 19.64852, -72.28838 19.6..."


In [135]:
# need to trim each of the values in stnode and endnode
service0001_edges_gdf['stnode'] = service0001_edges_gdf.stnode.apply(lambda x: x[6:])
service0001_edges_gdf['endnode'] = service0001_edges_gdf.endnode.apply(lambda x: x[6:])
service0002_edges_gdf['stnode'] = service0002_edges_gdf.stnode.apply(lambda x: x[6:])
service0002_edges_gdf['endnode'] = service0002_edges_gdf.endnode.apply(lambda x: x[6:])

In [136]:
service0001_edges_gdf.head()

,field_1,stnode,endnode,mode,length,trips,geometry
0,0,stop_0119,stop_0096,transit,2119.0,1,"LINESTRING (-71.73141 19.54854, -71.94963 19.6..."
1,1,stop_0143,stop_0260,transit,1462.0,1,"LINESTRING (-72.20758 19.74893, -72.27349 19.7..."
2,2,stop_0143,stop_0081,transit,74.0,2,"LINESTRING (-72.20758 19.74893, -72.20682 19.7..."
3,3,stop_0143,stop_0154,transit,122.0,1,"LINESTRING (-72.20758 19.74893, -72.20663 19.7..."
4,4,stop_0176,stop_0074,transit,706.0,1,"LINESTRING (-72.24224 19.78266, -72.22709 19.7..."


## Merge two graphs based on stnodes and endnodes

In [137]:
services_merged_df = pd.merge(service0001_edges_gdf, service0002_edges_gdf,  how='left', left_on=['stnode', 'endnode'], right_on = ['stnode', 'endnode'])

In [138]:
services_merged_df.head()

,field_1_x,stnode,endnode,mode_x,length_x,trips_x,geometry_x,field_1_y,mode_y,trips_y,length_y,geometry_y
0,0,stop_0119,stop_0096,transit,2119.0,1,"LINESTRING (-71.73141 19.54854, -71.94963 19.6...",157,transit,1,2119.0,"LINESTRING (-71.73141 19.54854, -71.94963 19.6..."
1,1,stop_0143,stop_0260,transit,1462.0,1,"LINESTRING (-72.20758 19.74893, -72.27349 19.7...",188,transit,1,1462.0,"LINESTRING (-72.20758 19.74893, -72.27349 19.7..."
2,2,stop_0143,stop_0081,transit,74.0,2,"LINESTRING (-72.20758 19.74893, -72.20682 19.7...",189,transit,6,121.16666666666667,"LINESTRING (-72.20758 19.74893, -72.20682 19.7..."
3,3,stop_0143,stop_0154,transit,122.0,1,"LINESTRING (-72.20758 19.74893, -72.20663 19.7...",190,transit,1,122.0,"LINESTRING (-72.20758 19.74893, -72.20663 19.7..."
4,4,stop_0176,stop_0074,transit,706.0,1,"LINESTRING (-72.24224 19.78266, -72.22709 19.7...",228,transit,3,635.3333333333334,"LINESTRING (-72.24224 19.78266, -72.22709 19.7..."


### convert key fields to numeric

In [139]:
services_merged_df['length_x'] = services_merged_df['length_x'].apply(pd.to_numeric, errors='coerce')
services_merged_df['length_y'] = services_merged_df['length_y'].apply(pd.to_numeric, errors='coerce')

services_merged_df['trips_x'] = services_merged_df['trips_x'].apply(pd.to_numeric, errors='coerce')
services_merged_df['trips_y'] = services_merged_df['trips_y'].apply(pd.to_numeric, errors='coerce')

### Calculate differences in length and number of trips

In [140]:
services_merged_df["length_diff"] = abs(services_merged_df.length_x - services_merged_df.length_y)
services_merged_df["trips_diff"] = services_merged_df.trips_y - services_merged_df.trips_x

In [141]:
services_merged_df.dtypes

field_1_x        object
stnode           object
endnode          object
mode_x           object
length_x        float64
trips_x           int64
geometry_x     geometry
field_1_y        object
mode_y           object
trips_y           int64
length_y        float64
geometry_y     geometry
length_diff     float64
trips_diff        int64
dtype: object

In [142]:
services_merged_df

,field_1_x,stnode,endnode,mode_x,length_x,trips_x,geometry_x,field_1_y,mode_y,trips_y,length_y,geometry_y,length_diff,trips_diff
0,0,stop_0119,stop_0096,transit,2119.0,1,"LINESTRING (-71.73141 19.54854, -71.94963 19.6...",157,transit,1,2119.000000,"LINESTRING (-71.73141 19.54854, -71.94963 19.6...",0.000000,0
1,1,stop_0143,stop_0260,transit,1462.0,1,"LINESTRING (-72.20758 19.74893, -72.27349 19.7...",188,transit,1,1462.000000,"LINESTRING (-72.20758 19.74893, -72.27349 19.7...",0.000000,0
2,2,stop_0143,stop_0081,transit,74.0,2,"LINESTRING (-72.20758 19.74893, -72.20682 19.7...",189,transit,6,121.166667,"LINESTRING (-72.20758 19.74893, -72.20682 19.7...",47.166667,4
3,3,stop_0143,stop_0154,transit,122.0,1,"LINESTRING (-72.20758 19.74893, -72.20663 19.7...",190,transit,1,122.000000,"LINESTRING (-72.20758 19.74893, -72.20663 19.7...",0.000000,0
4,4,stop_0176,stop_0074,transit,706.0,1,"LINESTRING (-72.24224 19.78266, -72.22709 19.7...",228,transit,3,635.333333,"LINESTRING (-72.24224 19.78266, -72.22709 19.7...",70.666667,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,337,stop_0159,stop_0125,transit,177.0,1,"LINESTRING (-72.17812 19.69676, -72.18074 19.7...",209,transit,3,158.333333,"LINESTRING (-72.17812 19.69676, -72.18074 19.7...",18.666667,2
338,338,stop_0156,stop_0123,transit,78.0,1,"LINESTRING (-72.19765 19.62419, -72.19785 19.6...",206,transit,3,52.333333,"LINESTRING (-72.19765 19.62419, -72.19785 19.6...",25.666667,2
339,339,stop_0124,stop_0035,transit,122.0,1,"LINESTRING (-72.15876 19.69403, -72.14931 19.6...",162,transit,3,139.000000,"LINESTRING (-72.15876 19.69403, -72.14931 19.6...",17.000000,2
340,340,stop_0132,stop_0046,transit,364.0,2,"LINESTRING (-72.20224 19.75554, -72.20458 19.7...",175,transit,4,314.750000,"LINESTRING (-72.20224 19.75554, -72.20458 19.7...",49.250000,2


In [143]:
# can only have one geometry column
services_merged_df.drop('geometry_y', inplace=True, axis=1)

In [144]:
services_merged_df.rename(columns={'geometry_x': 'geometry'}, inplace=True)

In [146]:
# save
services_merged_df.to_file(f"temp/edges_services_diff.shp")